<a href="https://colab.research.google.com/github/mahesh-tippanu/FakeReviewAI/blob/develop/fake_product_review_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np #importing the numpy module which we will be using in this project
import pandas as pd #importing the pandas module which will be used in this porject
import string#importing the pandas module which will be used in this porject
from sklearn.model_selection import train_test_split, GridSearchCV#importing the test_train_split module which will be used in this porject
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score #importing the classification report adn the confusion matrix module which will be used in this porject
import nltk#importing the nltk module which will be used in this porject
from nltk.corpus import stopwords#importing the nltk.corpus.stopwords module which will be used in this porject
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer#importing the extraction.text.CountVectorizer and TfidfTransformer module which will be used in this porject
from sklearn.pipeline import Pipeline#importing the sklearn.pipeline.Pipeline module which will be used in this porject
from sklearn.ensemble import RandomForestClassifier#importing the sklearn.ensemble.RandomForestClassifier module which will be used in this porject
from sklearn.svm import SVC#importing the sklearn.svm.SVC module which will be used in this porject
from sklearn.linear_model import LogisticRegression#importing the sklearn.linear_model.LogisticRegression module which will be used in this porject

In [7]:
from google.colab import files
import pandas as pd

# Upload the file
uploaded = files.upload()

# Automatically detect and load the uploaded file into a DataFrame
for file_name in uploaded.keys():
    print(f"Uploaded file: {file_name}")
    if file_name.endswith('.csv'):
        df = pd.read_csv(file_name)  # Load CSV into a DataFrame
        print("DataFrame loaded from CSV:")
        print(df.head())  # Show the first few rows
    elif file_name.endswith('.xlsx'):
        df = pd.read_excel(file_name)  # Load Excel file into a DataFrame
        print("DataFrame loaded from Excel file:")
        print(df.head())  # Show the first few rows
    else:
        print("Unsupported file format. Please upload a CSV or Excel file.")


Saving dataset.csv to dataset (3).csv
Uploaded file: dataset (3).csv
DataFrame loaded from CSV:
   Unnamed: 0            category  rating label  \
0           0  Home_and_Kitchen_5     5.0    CG   
1           1  Home_and_Kitchen_5     5.0    CG   
2           2  Home_and_Kitchen_5     5.0    CG   
3           3  Home_and_Kitchen_5     1.0    CG   
4           4  Home_and_Kitchen_5     5.0    CG   

                                              text_  
0  love well made sturdi comfort i love veri pretti  
1    love great upgrad origin i 've mine coupl year  
2      thi pillow save back i love look feel pillow  
3             miss inform use great product price i  
4       veri nice set good qualiti we set two month  


In [9]:
# Drop the 'UNNAMED' column if it exists in the DataFrame
if 'UNNAMED' in df.columns:
    df = df.drop(columns=['UNNAMED'])
    print("Dropped the 'UNNAMED' column.")
elif any(col.startswith('Unnamed') for col in df.columns):
    # Handle cases like 'Unnamed: 0', 'Unnamed: 1', etc.
    unnamed_cols = [col for col in df.columns if col.startswith('Unnamed')]
    df = df.drop(columns=unnamed_cols)
    print(f"Dropped unnamed columns: {unnamed_cols}")
else:
    print("No 'UNNAMED' column found.")

# Display the updated DataFrame
print(df.head())


Dropped unnamed columns: ['Unnamed: 0']
             category  rating label  \
0  Home_and_Kitchen_5     5.0    CG   
1  Home_and_Kitchen_5     5.0    CG   
2  Home_and_Kitchen_5     5.0    CG   
3  Home_and_Kitchen_5     1.0    CG   
4  Home_and_Kitchen_5     5.0    CG   

                                              text_  
0  love well made sturdi comfort i love veri pretti  
1    love great upgrad origin i 've mine coupl year  
2      thi pillow save back i love look feel pillow  
3             miss inform use great product price i  
4       veri nice set good qualiti we set two month  


In [11]:
# Drop the 'UNNAMED' column if it exists in the DataFrame
if 'UNNAMED' in df.columns:
    df = df.drop(columns=['UNNAMED'])
    print("Dropped the 'UNNAMED' column.")
elif any(col.startswith('Unnamed') for col in df.columns):
    # Handle cases like 'Unnamed: 0', 'Unnamed: 1', etc.
    unnamed_cols = [col for col in df.columns if col.startswith('Unnamed')]
    df = df.drop(columns=unnamed_cols)
    print(f"Dropped unnamed columns: {unnamed_cols}")
else:
    print("No 'UNNAMED' column found.")

# Print the dataset again after dropping the column
print("Updated dataset:")
print(df)


No 'UNNAMED' column found.
Updated dataset:
                           category  rating label  \
0                Home_and_Kitchen_5     5.0    CG   
1                Home_and_Kitchen_5     5.0    CG   
2                Home_and_Kitchen_5     5.0    CG   
3                Home_and_Kitchen_5     1.0    CG   
4                Home_and_Kitchen_5     5.0    CG   
...                             ...     ...   ...   
40427  Clothing_Shoes_and_Jewelry_5     4.0    OR   
40428  Clothing_Shoes_and_Jewelry_5     5.0    CG   
40429  Clothing_Shoes_and_Jewelry_5     2.0    OR   
40430  Clothing_Shoes_and_Jewelry_5     1.0    CG   
40431  Clothing_Shoes_and_Jewelry_5     5.0    OR   

                                                   text_  
0       love well made sturdi comfort i love veri pretti  
1         love great upgrad origin i 've mine coupl year  
2           thi pillow save back i love look feel pillow  
3                  miss inform use great product price i  
4            veri nice s

In [15]:
# Drop all rows with null (NaN) values
df = df.dropna()


# Print the updated dataset
print("Dataset after dropping all null rows:")
print(df)
 #dropping alll the null rows in the dataset



Dataset after dropping all null rows:
                           category  rating label  \
0                Home_and_Kitchen_5     5.0    CG   
1                Home_and_Kitchen_5     5.0    CG   
2                Home_and_Kitchen_5     5.0    CG   
3                Home_and_Kitchen_5     1.0    CG   
4                Home_and_Kitchen_5     5.0    CG   
...                             ...     ...   ...   
40427  Clothing_Shoes_and_Jewelry_5     4.0    OR   
40428  Clothing_Shoes_and_Jewelry_5     5.0    CG   
40429  Clothing_Shoes_and_Jewelry_5     2.0    OR   
40430  Clothing_Shoes_and_Jewelry_5     1.0    CG   
40431  Clothing_Shoes_and_Jewelry_5     5.0    OR   

                                                   text_  
0       love well made sturdi comfort i love veri pretti  
1         love great upgrad origin i 've mine coupl year  
2           thi pillow save back i love look feel pillow  
3                  miss inform use great product price i  
4            veri nice set goo

In [16]:
# Assuming the text column is named 'text', replace 'text' with your actual column name
if 'text' in df.columns:
    df['length'] = df['text'].apply(len)
    print("Added a 'length' column with the length of text.")
else:
    print("The column 'text' does not exist in the dataset.")

# Display the updated dataset
print(df.head())
 #storing the length of all the text into a separate column called 'length'

The column 'text' does not exist in the dataset.
             category  rating label  \
0  Home_and_Kitchen_5     5.0    CG   
1  Home_and_Kitchen_5     5.0    CG   
2  Home_and_Kitchen_5     5.0    CG   
3  Home_and_Kitchen_5     1.0    CG   
4  Home_and_Kitchen_5     5.0    CG   

                                              text_  
0  love well made sturdi comfort i love veri pretti  
1    love great upgrad origin i 've mine coupl year  
2      thi pillow save back i love look feel pillow  
3             miss inform use great product price i  
4       veri nice set good qualiti we set two month  


Let's extract the largest review...

In [17]:
import pandas as pd
from collections import Counter
import re
from nltk.corpus import stopwords
import nltk

# Download NLTK stopwords (run this once)
nltk.download('stopwords')

# Example: Assuming 'reviews' is the column with fake reviews
if 'reviews' in df.columns:
    # Preprocess text: Remove punctuation, convert to lowercase
    def preprocess_text(text):
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        text = text.lower()  # Convert to lowercase
        return text

    # Apply preprocessing to the 'reviews' column
    df['cleaned_reviews'] = df['reviews'].apply(preprocess_text)

    # Tokenize and flatten all words from fake reviews
    all_words = [word for review in df['cleaned_reviews'] for word in review.split()]

    # Filter out stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in all_words if word not in stop_words]

    # Count word frequencies
    word_freq = Counter(filtered_words)

    # Get the most common words
    most_common_words = word_freq.most_common(20)  # Top 20 words
    print("Most common words in fake reviews:")
    print(most_common_words)

    # Optionally, store the results in a new DataFrame
    common_words_df = pd.DataFrame(most_common_words, columns=['Word', 'Frequency'])
else:
    print("Column 'reviews' does not exist in the dataset.")


Column 'reviews' does not exist in the dataset.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [18]:
def convertmyTxt(rv): #here we are defining a function
    np = [c for c in rv if c not in string.punctuation] #this function is checking if it is present in punctuation or not.
    np = ''.join(np) #the character which are not in punctuation, we are storing them in a separate string
    return [w for w in np.split() if w.lower() not in stopwords.words('english')] #here we are returning a list of words from the sentences we just made in above line and checking if it is not a stopword

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming your data is in a CSV file named 'your_data.csv'
# Replace 'your_data.csv' with the actual file name
dataframe = pd.read_csv('dataset.csv')

# Splitting the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
    dataframe['text_'],  # Replace 'text_' with the actual text column name
    dataframe['label'],  # Replace 'label' with the actual label column name
    test_size=0.25,      # 25% for testing
    random_state=42      # Ensures reproducibility
)

# Display the shapes of the resulting splits
print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

x_train shape: (30324,)
x_test shape: (10108,)
y_train shape: (30324,)
y_test shape: (10108,)


In [22]:
pip = Pipeline([
    ('bow',CountVectorizer(analyzer=convertmyTxt)),
    ('tfidf',TfidfTransformer()),
    ('classifier',RandomForestClassifier())
]) #here we are defining our Random Forest Classifier model in which we will pass the training and testing data

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Replace NaN values with an empty string in training and testing data
x_train = x_train.fillna("")
x_test = x_test.fillna("")

# Initialize the vectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer to the training data and transform it
x_train_vec = vectorizer.fit_transform(x_train)

# Transform the test data using the fitted vectorizer
x_test_vec = vectorizer.transform(x_test)

print("Vectorization complete!")


Vectorization complete!


In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Assuming you already have x_train, x_test, y_train, y_test split earlier

# Handle NaN values by replacing them with empty strings (if not done already)
x_train = x_train.fillna("")
x_test = x_test.fillna("")

# Initialize the vectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer to the training data and transform it
x_train_vec = vectorizer.fit_transform(x_train)

# Transform the test data using the fitted vectorizer
x_test_vec = vectorizer.transform(x_test)

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Train the model with the transformed training data
rf_classifier.fit(x_train_vec, y_train)

# Predict the labels for the test set
y_pred = rf_classifier.predict(x_test_vec)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the Random Forest Classifier: {:.2f}%".format(accuracy * 100))

# Additional evaluation (optional)
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy of the Random Forest Classifier: 86.18%
Classification Report:
              precision    recall  f1-score   support

          CG       0.83      0.90      0.87      5038
          OR       0.90      0.82      0.86      5070

    accuracy                           0.86     10108
   macro avg       0.86      0.86      0.86     10108
weighted avg       0.86      0.86      0.86     10108



In [30]:
from sklearn.metrics import accuracy_score
import numpy as np

# Assuming you already have y_test and y_pred
y_pred = rf_classifier.predict(x_test_vec)

# Calculate the accuracy and print it
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of the model: ', str(np.round(accuracy * 100, 2)) + '%')
#here we are predicting the accuracy of the Random Forest Classifier model

Accuracy of the model:  86.18%


In [31]:
pip = Pipeline([
    ('bow',CountVectorizer(analyzer=convertmyTxt)),
    ('tfidf',TfidfTransformer()),
    ('classifier',SVC())
])#here we are defining our Support Vector Classifier model in which we will pass the training and testing data

In [32]:
pip.fit(x_train,y_train)#here we are passing the testing and training data into Random Forest Classifier

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function convertmyTxt at 0x781659587be0>)),
                ('tfidf', TfidfTransformer()), ('classifier', SVC())])

In [33]:
supportVectorClassifier = pip.predict(x_test)#here we are predicting the accuracy of the Random Forest Classifier model
supportVectorClassifier

array(['CG', 'CG', 'OR', ..., 'CG', 'OR', 'CG'], dtype=object)

In [34]:
print('accuracy of the model:',str(np.round(accuracy_score(y_test,supportVectorClassifier)*100,2)) + '%')#here we are predicting the accuracy of the Random Forest Classifier model

accuracy of the model: 88.13%


In [37]:
import re
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

def text_process(mess):#creating a function
    """
    1.remove punc
    2.remove stop words
    3.return list of clean text words

    """
    nopunc = [char for char in mess if char not in string.punctuation]#removing the punctuation marks and storing it in nopunc
    nopunc = ''.join(nopunc)#joining the nopunc



    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression

pip_lr = Pipeline([ # Change variable name to avoid overwriting previous pipeline
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf',TfidfTransformer()),
    ('classifier',LogisticRegression())
])#here we are defining our Logistic Regression model in which we will pass the training and testing data

In [38]:
pip.fit(x_train,y_train)#here we are passing the testing and training data into Random Forest Classifier

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function convertmyTxt at 0x781659587be0>)),
                ('tfidf', TfidfTransformer()), ('classifier', SVC())])

In [39]:
logisticRegression = pip.predict(x_test)#here we are predicting the accuracy of the Random Forest Classifier model
logisticRegression

array(['CG', 'CG', 'OR', ..., 'CG', 'OR', 'CG'], dtype=object)

In [40]:
print('accuracy of the model:',str(np.round(accuracy_score(y_test,logisticRegression)*100,2)) + '%')#here we are predicting the accuracy of the Random Forest Classifier model

accuracy of the model: 88.13%
